In [1]:
from notebook.services.config import ConfigManager
cfgm = ConfigManager()
cfgm.update('livereveal', {
              'theme': 'simple',
              'transition': 'convex',
              'start_slideshow_at': 'selected'
});

In [2]:
from IPython.core.display import HTML
HTML("""<style>
.answers { 
    visibility: hidden;
}
</style>""")

# Python Emissions Perturbation
Author: Barron H. Henderson

# Prepare by getting data

In [3]:
%pylab inline
%cd working

Populating the interactive namespace from numpy and matplotlib
/Users/barronh/Development/RAQMSandPython/working


In [4]:
from urllib.request import urlretrieve, url2pathname
import os
qemisurl = 'ftp://data.as.essie.ufl.edu/pub/exch/CMAQandPython/cmaq/emis_mole_all_20060801_12US1_cmaq_cb05_tx_C25_2006am.nc4'
qemisoldpath = os.path.join('cmaq', os.path.basename(qemisurl))
urlretrieve(qemisurl, filename = qemisoldpath)

('cmaq/emis_mole_all_20060801_12US1_cmaq_cb05_tx_C25_2006am.nc4',
 <email.message.Message at 0x10d2b2828>)

# CMAQ: Emissions Perturbation

1. Copy a standard emissions input file
2. Open the file in edit mode
3. Change the concentration (e.g., cut NOx by half)
4. Close the file

In [4]:
emisoldpath = 'cmaq/emis_mole_all_20060801_12US1_cmaq_cb05_tx_C25_2006am.nc4'
emisnewpath = 'emis_cmaq_new.nc'

# CMAQ: Copy a standard emissions file

In [5]:
import shutil
shutil.copyfile(emisoldpath, emisnewpath)

'emis_cmaq_new.nc'

# CMAQ: Open the new file in edit mode

In [6]:
from PseudoNetCDF import PNC
emisargs = PNC('--format=netcdf,mode="r+"', emisnewpath)
emisnewfile = emisargs.ifiles[0]
emisnewfile.variables['NO'][:] /= 2
emisnewfile.variables['NO2'][:] /= 2
emisnewfile.history = getattr(emisnewfile, 'history', '')
emisnewfile.history += 'NOx / 2 everywhere; '
emisnewfile.close()

# CMAQ: reopen and verify

In [10]:
emisargs = PNC("--format=netcdf", emisoldpath, emisnewpath)
oldfile, newfile = emisargs.ifiles
print('SPC    OLD    NEW UNITS')
Mw = dict(NO = 30, NO2 = 46) # g/mole
t2h = 3600
g2Mg = 1e-6
for k in 'NO NO2'.split():
    old = oldfile.variables[k][:].sum()*t2h*g2Mg*Mw[k]
    new = newfile.variables[k][:].sum()*t2h*g2Mg*Mw[k]
    print('%3s %6.1f %6.1f Mg/day' % (k, old, new))

print(newfile.history)
newfile.close()

SPC    OLD    NEW UNITS
 NO 5127.8 3449.7 Mg/day
NO2  809.8  544.8 Mg/day
emis_cmaq_new.nc;--format=netcdf,mode="r+" emis_cmaq_new.nc;NOx / 2 everywhere; emis_cmaq_new.nc;--format=netcdf,mode="r+" emis_cmaq_new.nc;Updating NOx (*=100) at i=91.000000 j=99 *= 10; Updating NOx (*=100) at i=54.000000 j=73 *= 10; 


# And (or) at specific sites

In [9]:
from PseudoNetCDF.coordutil import getproj4
import pyproj
lons = [-79.0558, -84.388]
lats = [35.9132, 33.7499]
proj4str = getproj4(oldfile, withgrid = True)
print(proj4str)
proj = pyproj.Proj(proj4str, preserve_units = True)
emisargs = PNC('--format=netcdf,mode="r+"', emisnewpath)
emisnewfile = emisargs.ifiles[0]
emisnewfile.history = getattr(emisnewfile, 'history', '')
for lon, lat in zip(lons, lats):
    x, y = proj(lon, lat)
    i, j = np.int64(x), np.int64(y)
    print(lon, lat)
    emisnewfile.history += 'Updating NOx (*=100) at i=%f j=%d *= 10; ' % (i,j)
    emisnewfile.variables['NO'][:, :, j, i] *= 100
    emisnewfile.variables['NO2'][:, :, j, i] *= 100

emisnewfile.close()

+proj=lcc +a=6370000.0 +b=6370000.0 +lon_0=-97.0 +lat_1=33.0 +lat_2=45.0 +lat_0=40.0 +x_0=-504000.0 +y_0=1488000.0 +to_meter=12000.0m +no_defs
-79.0558 35.9132
-84.388 33.7499


# Were site-wide and specific site changes additive? (answer hidden)
<div class="answers">
Yes. Files can be iteratively opened and modified. It is good to edit the history so you can check!

1. You can check by rerunning confirmation of mass update.
2. You can check via history property.
</div>

# CAMx: Emissions Perturbation

Repeat the CMAQ process with two exceptions:
1. use uamiv instead of netcdf
2. remove the change t2h from 3600 (what should it be?)

In [ ]:
from urllib.request import urlretrieve, url2pathname
import os
xemisurl = 'ftp://data.as.essie.ufl.edu/pub/exch/CMAQandPython/camx/emiss.stl.36km.20020603.a1.bin'
xemisoldpath = os.path.join('camx', os.path.basename(xemisurl))
urlretrieve(xemisurl, filename = xemisoldpath)